In [1]:
import os
import sys
sys.path.append('../')
sys.path.append('../train')
os.environ['CUDA_VISIBLE_DEVICES']='5'
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
import cv2
from PIL import Image
from glob import glob

In [2]:
from resnet import *
from dr_model import *

In [3]:
model = DRModel('rsn34', 1024, 2)

In [4]:
weights_file = './dr_cls_yyy/ct_pos_recognition_0020_best.pth'
model.load_state_dict(torch.load(weights_file))

<All keys matched successfully>

In [5]:
# def get_mask(img, model, alpha):
# #     model.eval()
#     img1 = img.copy()
#     img1 = np.array(img1, dtype=np.float)
#     input_img = torch.from_numpy(img1)

#     input_img = torch.unsqueeze(input_img, 0)
#     input_img = torch.unsqueeze(input_img, 0)
    
# #     input_img /= 255
    
#     weights=list(list(model.children())[-1].parameters())[0]
#     model_feat4 = torch.nn.Sequential(*list(model.children())[:-1])
    
#     model_feat4 = torch.nn.DataParallel(model_feat4).cuda()
#     model_feat4.eval()
#     img_feat4 = model_feat4(input_img.float().cuda())
#     model = torch.nn.DataParallel(model).cuda()
#     model.eval()
#     result = model(input_img.float().cuda())
#     result1 = torch.nn.Softmax()(result)
# #     print(result1)
# #     print(torch.argmax(result1))
#     position = torch.argmax(result1).cpu().detach().numpy()
    
#     attn_feat4 = torch.squeeze(img_feat4)
    
# #     print(img_feat4.shape)
# #     print(attn_feat4.shape)
# #     print(weights.shape)
#     heat_map = torch.matmul(attn_feat4.permute([1,2,0]), weights.transpose(1,0))
    
#     heat_map = heat_map.cpu().detach().numpy()
    
#     print(result1)
#     print(torch.argmax(result1).cpu().detach().numpy())
#     print(heat_map.shape)
# #     mask=cv2.resize(np.sum(heat_map, 2)/2, (1024,1024))
#     sub_heat_map = heat_map[:,:,torch.argmax(result1).cpu().detach().numpy()]
    
#     min_v = np.min(sub_heat_map)
#     max_v = np.max(sub_heat_map)
#     sub_heat_map = (sub_heat_map-min_v)/(max_v-min_v)
# #     sub_heat_map[sub_heat_map<0.8]=0
#     sub_heat_map = np.square(sub_heat_map)

#     mask=cv2.resize(sub_heat_map, (1024,1024))
    
# #     mask=cv2.resize(heat_map[:,:,torch.argmax(result1).cpu().detach().numpy()]/5, (1024,1024))
# #     mask=cv2.resize(heat_map[:,:,2]/5, (1024,1024))
    
#     heat_map = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
    
#     img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
# #     plt.figure()
# #     plt.imshow(img_rgb)
    
# #     attn_img = img_rgb*alpha+(1-alpha)*heat_map
#     attn_img = img_rgb+heat_map

#     return attn_img, heat_map, position

In [9]:
def get_mask(img, model, alpha, img_file):
#     model.eval()
    img1 = img.copy()
    img1 = np.array(img1, dtype=np.float)
    input_img = torch.from_numpy(img1)

    input_img = torch.unsqueeze(input_img, 0)
    input_img = torch.unsqueeze(input_img, 0)
    
#     input_img /= 255
    
    weights=list(list(model.children())[-1].parameters())[0]
    model_feat4 = torch.nn.Sequential(*list(model.children())[:-1])
    
    model_feat4 = torch.nn.DataParallel(model_feat4).cuda()
    model_feat4.eval()
    img_feat4 = model_feat4(input_img.float().cuda())
    model = torch.nn.DataParallel(model).cuda()
    model.eval()
    result = model(input_img.float().cuda())
    result1 = torch.nn.Softmax()(result)
#     print(result1)
#     print(torch.argmax(result1))
    position = torch.argmax(result1).cpu().detach().numpy()
    
    attn_feat4 = torch.squeeze(img_feat4)
    
#     print(img_feat4.shape)
#     print(attn_feat4.shape)
#     print(weights.shape)


    output = img_feat4
    weights1 = list(model_feat4.parameters())[-2][1]
    heatmap = None
    print('xxx')
    print(weights1.shape)
    print(heatmap.shape)
    for i in range (0, len(weights1)):
        map = output[0,i,:,:]
        if i == 0: heatmap = weights1[i] * map
        else: heatmap += weights1[i] * map

    npHeatmap = heatmap.cpu().data.numpy()
    cam = npHeatmap / np.max(npHeatmap)
    transCrop = 1024
    cam = cv2.resize(cam, (transCrop, transCrop))
    heatmap = cv2.applyColorMap(np.uint8(255*cam), cv2.COLORMAP_JET)
#     imgOriginal = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    imgOriginal = cv2.imread(img_file, 1)
    imgOriginal = cv2.resize(imgOriginal, (transCrop, transCrop))
    imgx = heatmap * 0.5 + imgOriginal
    
    

    heat_map = torch.matmul(attn_feat4.permute([1,2,0]), weights.transpose(1,0))
    
    heat_map = heat_map.cpu().detach().numpy()
    
    print(result1)
    print(torch.argmax(result1).cpu().detach().numpy())
    print(heat_map.shape)
#     mask=cv2.resize(np.sum(heat_map, 2)/2, (1024,1024))
    sub_heat_map = heat_map[:,:,torch.argmax(result1).cpu().detach().numpy()]
    
    min_v = np.min(sub_heat_map)
    max_v = np.max(sub_heat_map)
    sub_heat_map = (sub_heat_map-min_v)/(max_v-min_v)
#     sub_heat_map[sub_heat_map<0.8]=0
    sub_heat_map = np.square(sub_heat_map)

    mask=cv2.resize(sub_heat_map, (1024,1024))
    
#     mask=cv2.resize(heat_map[:,:,torch.argmax(result1).cpu().detach().numpy()]/5, (1024,1024))
#     mask=cv2.resize(heat_map[:,:,2]/5, (1024,1024))
    
    heat_map = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
    
    img_rgb = cv2.cvtColor(img, cv2.COLOR_GRAY2RGB)
    
#     plt.figure()
#     plt.imshow(img_rgb)
    
#     attn_img = img_rgb*alpha+(1-alpha)*heat_map
    attn_img = img_rgb+heat_map

    return imgx, heatmap, position

In [10]:
def plot_attention_area(img_file):
    img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
#     img = cv2.imread(img_file, cv2.IMREAD_GRAYSCALE)
    img = cv2.resize(img, (1024,1024))
    
    
    alpha = 0.5
    attn_img, mask, position = get_mask(img, model, alpha, img_file)
    
#     if position != 1 and position != 0:
#         return

    plt.figure(figsize=(15,10))
    plt.subplot(1,3,1)
    plt.imshow(attn_img)
    plt.subplot(1,3,2)
    plt.imshow(mask)
    plt.subplot(1,3,3)
    plt.xlabel('position:{}'.format(position))
    img = cv2.cvtColor(img, cv2.COLOR_GRAY2BGR)
    plt.imshow(img)
    plt.show()

In [11]:
img_file = '/data/zhangwd/data/external/xray/openi/data/CXR2041_IM-0685-3001.png'
plot_attention_area(img_file)

xxx
torch.Size([])


/home/zhangwd/.conda/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:21: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.


AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
def plot_cam(config_file):
    with open(config_file,encoding='gb2312') as f:
        config = json.load(f)
    val_file = config["test_list_file"]
    pos_files  = []
    neg_files = []
    with open(val_file) as f:
        for line in f.readlines():
            line = line.strip()
            if line is None or len(line) == 0:
                continue
            ss = line.split('\t')
            if len(ss) != 2:
                continue
            if int(ss[1]) == 1:
                pos_files.append(ss[0])
            else:
                neg_files.append(ss[0])
    for f in pos_files[:20]:
        plot_attention_area(f)
#     for f in neg_files[:20]:
#         plot_attention_area(f)

In [ ]:
plot_cam('./config/config_dr_xiongqiangjiye.json')

In [ ]:
mask = np.arange(256)
mask = np.reshape(mask, [16,16])
mask = mask/255
# mask = np.square(mask)
heat_map = cv2.applyColorMap(np.uint8(255*mask), cv2.COLORMAP_JET)
plt.figure()
plt.imshow(heat_map)